# Get_Original_AlphaNet_Data

In [2]:
Alpha_Name = "AlphaNet_Original_Input_12.22_1d_return_normalized"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "1d_ret"

In [3]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp

# Get_Original_AlphaNet_Data

In [4]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.makedirs(path+i)

In [5]:
df = pd.read_csv(path+"data.csv")
df.dropna(inplace=True,axis=0,how="any")
df

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.018297
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.023288
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.051118
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.028571
...,...,...,...,...,...,...,...,...,...,...,...,...
5357699,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,-0.040395
5357700,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,-0.005927
5357701,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.025238
5357702,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,-0.022042


In [6]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(one_data):
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [7]:
if __name__=="__main__":
    pool = mp.Pool(10)
    result = {}
    t1 = time.time()
    grouped = df.groupby("ticker")
    for ticker , one_data in tqdm(grouped):
        result[ticker] = (pool.apply_async(test, args=(one_data,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 146644.95it/s]

并行执行时间： 48


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,open,...,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,1126.20,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,4.3668,1105.60,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,3.4269,1101.33,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,2.8375,1058.70,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,5.0564,1056.57,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,688981,54.50,55.95,54.50,55.82,55.353800,209785.85,0.6310,1.9147,1.9147,56.70,...,58.53,59.11,57.55,57.70,58.173300,273520.23,-1.4349,2.4964,2.4964,0.036184
2021-06-28 09:30:00,688981,56.70,56.98,55.60,55.86,56.113100,260448.82,0.0717,2.3771,2.3771,55.62,...,57.76,59.99,57.76,59.31,59.274600,474988.89,2.7903,4.3352,4.3352,-0.012531
2021-06-29 09:30:00,688981,55.62,56.40,55.05,55.08,55.686500,203770.43,-1.3963,1.8598,1.8598,55.19,...,59.85,60.19,58.86,59.06,59.476200,341515.10,-0.4215,3.1170,3.1170,0.009983


In [8]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.dropna(axis=0,inplace=True) # delete none
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,261,262,263,264,265,266,267,268,269,target
0,2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
1,2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2,2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
3,2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
4,2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230320,2021-06-25 09:30:00,688981,54.50,55.95,54.50,55.82,55.353800,209785.85,0.6310,1.9147,...,58.53,59.11,57.55,57.70,58.173300,273520.23,-1.4349,2.4964,2.4964,0.036184
5230321,2021-06-28 09:30:00,688981,56.70,56.98,55.60,55.86,56.113100,260448.82,0.0717,2.3771,...,57.76,59.99,57.76,59.31,59.274600,474988.89,2.7903,4.3352,4.3352,-0.012531
5230322,2021-06-29 09:30:00,688981,55.62,56.40,55.05,55.08,55.686500,203770.43,-1.3963,1.8598,...,59.85,60.19,58.86,59.06,59.476200,341515.10,-0.4215,3.1170,3.1170,0.009983
5230323,2021-06-30 09:30:00,688981,55.19,55.19,54.49,54.99,54.795800,121147.00,-0.1634,1.1057,...,59.10,60.08,58.70,59.25,59.354400,291408.81,0.3217,2.6597,2.6597,0.026973


In [9]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(10)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 3726.49it/s]


Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2015-09-01_2016-01-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2016-09-01_2017-01-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2017-06-01_2017-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Data/2016-03-01_2016-06-01.par
S

# Get Factor

In [10]:
day = 30
stride = 10
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(data_length, stride):
    if data_length % stride == 0:
        step_list = list(range(0, data_length + stride, stride))
    elif data_length % stride <= 5:
        mod = data_length % stride
        step_list = list(range(0, data_length - stride, stride)) + [data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
    return step_list

In [11]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 167772.16it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [12]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov") + name)
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
#     print("read" + input_path + name)
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(data_length, stride)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = cov(x,num,num_rev,step_list)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [13]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

  0%|          | 0/3 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Cov/2015-01-01_2015-03-01.par


 67%|██████▋   | 2/3 [00:08<00:04,  4.39s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Cov/2016-01-01_2016-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Cov/2017-01-01_2017-03-01.par


 67%|██████▋   | 2/3 [00:06<00:03,  3.23s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2019-09-01_2020-01-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2021-01-01_2021-03-01.par


 67%|██████▋   | 2/3 [00:00<00:00,  4.04it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2020-03-01_2020-06-01.par


100%|██████████| 3/3 [00:00<00:00,  3.84it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2020-06-01_2020-09-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Decay/2020-09-01_2021-01-01.par
并行执行时间： 89


In [14]:
t1 = time.time()
pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 3/3 [00:01<00:00,  2.79it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2015-01-01_2015-03-01.par


100%|██████████| 3/3 [00:08<00:00,  2.96s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2017-01-01_2017-03-01.par


 67%|██████▋   | 2/3 [00:09<00:04,  4.75s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2021-01-01_2021-03-01.par


 67%|██████▋   | 2/3 [00:16<00:08,  8.62s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2020-06-01_2020-09-01.par


100%|██████████| 3/3 [00:25<00:00,  8.35s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Corr/2020-09-01_2021-01-01.par
并行执行时间： 78


# Feat Concat ( Save to Feat_Concat file)

In [21]:
os.mkdir(path+"Feat_Concat")

FileExistsError: [Errno 17] File exists: '/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat'

In [22]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 198999.82it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [23]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Corr','Std','Zscore','Return','Decay']

final = []
def test(alpha,dir_list):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
    print(path+"Feat_Concat/"+alpha)
    return None

for alpha in tqdm(path_list):
#     test(alpha,dir_list)
    pool.apply_async(test, args=(alpha,dir_list))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 45609.33it/s]


/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2015-01-01_2015-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2016-01-01_2016-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2017-01-01_2017-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2015-03-01_2015-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2016-06-01_2016-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2016-03-01_2016-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2018-01-01_2018-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Feat_Concat/2017-03-01_2017-06-01.par
/home/wuwenjun/Data

# Feat Daily Norm ( Save to Feat_Concat_Norm_Daily)

In [ ]:
# import time
# import os
# from Research.backtest.bt import BTDaily
# from Research.feature.ft import FeatureAnalysis
# import Research.utils.namespace as namespace
# import Research.utils.normalization as norm
# from Platform.database.mysql import MysqlAPI
# from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
# from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
# import DataAPI
# from os import walk
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from progressbar import ProgressBar
# from tqdm import tqdm
# Alpha_Name = "AlphaNet_Original_Input_12.14"
# path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
# target = "5d_ret"

In [ ]:
# os.mkdir(path+"Feat_Concat_Norm_Daily")

In [ ]:
# path_list = []
# data_path = path + "Data/"
# dataframe_list = pd.DataFrame()
# for f, _, i in walk(data_path):
#     for j in tqdm(i):
#         path_list.append(j)
# path_list

In [ ]:
# import multiprocessing as mp


# """ Multi Processing"""
# pool = mp.Pool(10)
# t1 = time.time()
# def test(file):
#     df = pd.read_parquet(path + "Feat_Concat/" + file)
#     final = []
#     for day in tqdm(df['timestamp'].drop_duplicates()):
#         array = df[df['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = array.index
        
#         # Batch
#         array = torch.from_numpy(np.array(array)).to(torch.float) 
#         batch = nn.BatchNorm1d(array.shape[1])
#         array = batch(array)
#         array = pd.DataFrame(array.detach().numpy())
#         array.index = index
#         final.append(array)
        
#     final = pd.concat(final,axis=0)
#     final.columns = [str(i) for i in range(final.shape[1])]
#     final.reset_index(inplace=True)
#     final.to_parquet(path + "Feat_Concat_Norm_Daily/" + file)
#     print("Save to : ", path + "Feat_Concat_Norm_Daily/" + file)
#     return None

# for file in path_list:
# #     test(file)
#     pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
# pool.close()
# pool.join()

# t2 = time.time()
# print("并行执行时间：", int(t2 - t1))

# Pooling ( Save to Pooling)

In [19]:
os.mkdir(path+"Pooling")

In [24]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 162279.62it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [26]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))
    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.columns = [str(i) for i in range(array.shape[1])]
    array.reset_index(inplace=True)
#     # Daily Norm
#     final = []
#     for day in tqdm(array['timestamp'].drop_duplicates()):
#         df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)

#     final = pd.concat(final,axis=0)
#     final.reset_index(inplace=True)
    array.to_parquet(path + "Pooling/" + file)
    print("Save to : ", path + "Pooling/" + file)
    return None
for file in path_list:
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Pooling/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_n

# Concat Input

In [27]:
os.mkdir(path+"Final")

In [28]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 164730.97it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [29]:
def test(file):
    array = pd.concat([pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"]),
                   pd.read_parquet(path + "Pooling/" + file).set_index(["timestamp","ticker"])],axis=1)
    array.columns = [str(i) for i in range(array.shape[1])]

    array.reset_index().to_parquet(path +"Final/" + file)
    print("Save to : ",path +"Final/" + file)
    return None

In [30]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 33492.60it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2018-01-01_2018-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/

In [31]:
pd.read_parquet(path + "Final/" + file)

,timestamp,ticker,0,1,2,3,4,5,6,7,...,638,639,640,641,642,643,644,645,646,647
0,2021-03-01 09:30:00,1,5.427717,6.912647,6.868007,5.795646,6.865963,6.895270,5.629119,6.921366,...,1.898789,1.305608,0.347633,-1.390788,-0.851789,-0.924843,-0.973580,-0.616744,-0.719087,-0.828046
1,2021-03-02 09:30:00,1,5.311970,7.194792,7.060676,5.770009,7.154916,7.090503,5.563545,7.232574,...,1.886291,1.430298,0.461538,-1.353464,-0.851204,-0.943749,-1.006649,-0.617727,-0.752986,-0.885682
2,2021-03-03 09:30:00,1,5.675701,7.509904,7.185215,6.169109,7.404818,7.235582,5.915178,7.437110,...,1.879865,1.426869,0.474791,-1.348303,-0.875612,-0.946934,-1.002095,-0.657302,-0.761409,-0.885794
3,2021-03-04 09:30:00,1,6.234763,7.728699,7.273295,6.532270,7.670158,7.358367,6.586197,7.854291,...,1.876673,1.424578,0.512365,0.024114,-0.806271,-0.899936,-0.983038,-0.535199,-0.682709,-0.857594
4,2021-03-05 09:30:00,1,6.556331,7.750983,7.234983,6.925138,7.629039,7.240884,6.973928,7.892092,...,1.878452,1.387472,0.488096,-0.549819,-0.826535,-0.922143,-1.012131,-0.558883,-0.718132,-0.903870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259327,2021-05-25 09:30:00,688981,-0.205759,-0.172828,-0.243548,-0.231791,-0.175337,-0.241100,-0.227810,-0.168173,...,0.641778,-0.012250,-0.463250,-1.302894,0.008192,-0.194833,-0.335853,-0.535158,-0.705576,-0.844803
259328,2021-05-26 09:30:00,688981,-0.205542,-0.163297,-0.249374,-0.230466,-0.165172,-0.250612,-0.225553,-0.157291,...,0.508771,0.012777,-0.549115,-1.219882,-0.035257,-0.215560,-0.397033,-0.562927,-0.713642,-0.871802
259329,2021-05-27 09:30:00,688981,-0.204947,-0.157472,-0.234763,-0.224758,-0.156012,-0.239171,-0.215811,-0.151094,...,0.702132,0.295046,-0.393439,-0.753359,-0.119430,-0.202577,-0.247783,-0.624603,-0.701687,-0.748747
259330,2021-05-28 09:30:00,688981,-0.200159,-0.158753,-0.226590,-0.217727,-0.157752,-0.231731,-0.207808,-0.153925,...,0.771803,0.685037,0.126723,-0.867258,0.146523,-0.040570,-0.181952,-0.447830,-0.584031,-0.693923


# Normalize Target and Concat

In [32]:
dataframe_list = pd.read_csv(path + "data.csv").set_index(["timestamp","ticker"])
df = dataframe_list[target].unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=["target"])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(["timestamp","ticker"],inplace=True)
df

target
timestamp           ticker          
2015-01-06 09:30:00 1      -1.212328
                    2      -1.374955
                    4       1.049421
                    5      -0.616170
                    6      -1.547116
...                              ...
2021-07-01 09:30:00 688700 -0.708172
                    688777  0.318826
                    688788 -0.256562
                    688819 -0.161340
                    688981 -0.073268

[5357704 rows x 1 columns]

In [33]:
def test(i):
    dataframe_list = pd.read_parquet(path +"Final/" + i).set_index(["timestamp","ticker"])
    dataframe_list = pd.merge(dataframe_list,df,how="left",left_index=True,right_index=True)
    dataframe_list.reset_index().to_parquet(path +"Final/" + i)
    print("Save to : ",path +"Final/" + i)
    return None

In [34]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 37957.50it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return_normalized/Final/

In [36]:
pd.read_parquet(path + "Final/" + file)

,timestamp,ticker,0,1,2,3,4,5,6,7,...,639,640,641,642,643,644,645,646,647,target
0,2021-03-01 09:30:00,1,5.427717,6.912647,6.868007,5.795646,6.865963,6.895270,5.629119,6.921366,...,1.305608,0.347633,-1.390788,-0.851789,-0.924843,-0.973580,-0.616744,-0.719087,-0.828046,-0.588642
1,2021-03-02 09:30:00,1,5.311970,7.194792,7.060676,5.770009,7.154916,7.090503,5.563545,7.232574,...,1.430298,0.461538,-1.353464,-0.851204,-0.943749,-1.006649,-0.617727,-0.752986,-0.885682,0.294105
2,2021-03-03 09:30:00,1,5.675701,7.509904,7.185215,6.169109,7.404818,7.235582,5.915178,7.437110,...,1.426869,0.474791,-1.348303,-0.875612,-0.946934,-1.002095,-0.657302,-0.761409,-0.885794,1.756611
3,2021-03-04 09:30:00,1,6.234763,7.728699,7.273295,6.532270,7.670158,7.358367,6.586197,7.854291,...,1.424578,0.512365,0.024114,-0.806271,-0.899936,-0.983038,-0.535199,-0.682709,-0.857594,0.234582
4,2021-03-05 09:30:00,1,6.556331,7.750983,7.234983,6.925138,7.629039,7.240884,6.973928,7.892092,...,1.387472,0.488096,-0.549819,-0.826535,-0.922143,-1.012131,-0.558883,-0.718132,-0.903870,-1.036358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259327,2021-05-25 09:30:00,688981,-0.205759,-0.172828,-0.243548,-0.231791,-0.175337,-0.241100,-0.227810,-0.168173,...,-0.012250,-0.463250,-1.302894,0.008192,-0.194833,-0.335853,-0.535158,-0.705576,-0.844803,-0.144216
259328,2021-05-26 09:30:00,688981,-0.205542,-0.163297,-0.249374,-0.230466,-0.165172,-0.250612,-0.225553,-0.157291,...,0.012777,-0.549115,-1.219882,-0.035257,-0.215560,-0.397033,-0.562927,-0.713642,-0.871802,-0.605477
259329,2021-05-27 09:30:00,688981,-0.204947,-0.157472,-0.234763,-0.224758,-0.156012,-0.239171,-0.215811,-0.151094,...,0.295046,-0.393439,-0.753359,-0.119430,-0.202577,-0.247783,-0.624603,-0.701687,-0.748747,0.484039
259330,2021-05-28 09:30:00,688981,-0.200159,-0.158753,-0.226590,-0.217727,-0.157752,-0.231731,-0.207808,-0.153925,...,0.685037,0.126723,-0.867258,0.146523,-0.040570,-0.181952,-0.447830,-0.584031,-0.693923,0.102041
